In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import re
import pandas as pd
import numpy as np
import shutil

In [3]:
TEMPORAL_PATH = "/content/drive/MyDrive/ADSDB/landing/temporal"
PERSISTENT_PATH = "/content/drive/MyDrive/ADSDB/landing/persistent"

INFLATION_FILENAME = "inflation_rate"
EMPLOYMENT_FILENAME = "employment_rate"
HPI_FILENAME = "house_price_index"
HPI_WEIGHTS_FILENAME = "house_price_index_weights"

FILENAMES = [INFLATION_FILENAME, EMPLOYMENT_FILENAME, HPI_FILENAME, HPI_WEIGHTS_FILENAME]

CSV_FORMAT = "csv"

In [4]:
def convert_period_to_year_quarter(df):
    roman_numerals = { 'I': 1, 'II': 2, 'III': 3, 'IV': 4 }

    # Replace Roman numerals with corresponding Arabic numerals
    try:
      df['Quarter'] = df['Periodo'].str.extract(r'Q([IVX]+)')
      df['Quarter'] = df['Quarter'].map(roman_numerals).astype(int)
    except:
      pass

    # Extract year and quarter
    try:
      df['Periodo'] = df['Periodo'].astype(str).str.extract(r'(\d{4})').astype(int)
      df = df.rename(columns={'Periodo': 'Year'})
    except:
      pass

    # Convert date to standard format
    try:
      df['date'] = df['date'].str.extract(r'(\d{4})').astype(int)
      df = df.rename(columns={'date': 'Year'})
    except:
      pass

    return df

In [5]:
def clean_column_names(df):
    # Clean column names
    df.columns = [re.sub(r'[()]|\[\]', '', col).strip().lower().replace('%', 'percent').replace(' ', '_') for col in df.columns]

    # Remove columns with empty names
    empty_columns = df.columns[df.columns == '']
    df = df.drop(empty_columns, axis=1)

    return df

In [6]:
def get_clean_df(filename, temporal_path=TEMPORAL_PATH, format=CSV_FORMAT, sep=';'):
  df = pd.read_csv(f'{temporal_path}/{filename}.{format}', sep=sep)
  df = convert_period_to_year_quarter(df)
  df = clean_column_names(df)
  return df

In [7]:
def get_year_range(filenames=FILENAMES):
    min_year = max(get_clean_df(filename)['year'].min() for filename in filenames)
    max_year = min(get_clean_df(filename)['year'].max() for filename in filenames)

    return (min_year, max_year + 1)

In [8]:
def create_folders(year_range, persistent_path=PERSISTENT_PATH):
  for year in range(year_range[0], year_range[1]):
    path = f'{persistent_path}/{year}'
    if os.path.isdir(path):
      shutil.rmtree(path)
    os.mkdir(path)

In [9]:
def save_csvs_per_year(df, filename, year_range, persistent_path=PERSISTENT_PATH, format=CSV_FORMAT):
  for year in range(year_range[0], year_range[1]):
    temp_df = df[df['year'] == year]
    temp_df.to_csv(f'{persistent_path}/{year}/{filename}_{year}.{format}', index=False)

In [10]:
def save_all_csvs(year_range, filenames=FILENAMES):
  [save_csvs_per_year(get_clean_df(filename), filename, year_range) for filename in filenames]

In [11]:
if __name__ == "__main__":
  year_range = get_year_range()
  create_folders(year_range)
  save_all_csvs(year_range)